In [1]:
from datasets.lineage_tracing import *

In [2]:
ltseq = LTSeqDataset()

--2025-04-02 10:45:18--  https://kleintools.hms.harvard.edu/paper_websites/state_fate2020/stateFate_inVitro_normed_counts.mtx.gz
Resolving kleintools.hms.harvard.edu (kleintools.hms.harvard.edu)... 134.174.159.103
Connecting to kleintools.hms.harvard.edu (kleintools.hms.harvard.edu)|134.174.159.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2065554808 (1.9G) [application/x-gzip]
Saving to: ‘stateFate_inVitro_normed_counts.mtx.gz’

     0K .......... .......... .......... .......... ..........  0% 4.82M 6m49s
    50K .......... .......... .......... .......... ..........  0% 4.93M 6m44s
   100K .......... .......... .......... .......... ..........  0%  107M 4m36s
   150K .......... .......... .......... .......... ..........  0% 3.40M 5m52s
   200K .......... .......... .......... .......... ..........  0% 16.0M 5m6s
   250K .......... .......... .......... .......... ..........  0% 5.74M 5m12s
   300K .......... .......... .......... .......... .........

In [4]:
ltseq.data

tensor([[[-9.4270e-01, -3.1793e+00, -1.7418e+00,  ...,  1.3530e-02,
           4.7961e-02,  7.4252e-01],
         [-1.0569e+00, -3.6084e+00, -1.1600e+00,  ..., -2.2412e-01,
           1.3439e-01, -3.7990e-01],
         [-5.6815e-01, -3.6929e+00, -1.7942e+00,  ..., -3.6483e-01,
           1.1169e+00,  2.4111e+00],
         ...,
         [-1.0569e+00, -3.6084e+00, -1.1600e+00,  ..., -2.2412e-01,
           1.3439e-01, -3.7990e-01],
         [-1.1598e+00, -4.2480e+00, -1.5038e+00,  ...,  5.7074e-01,
           7.6002e-01, -1.4566e-01],
         [-1.2674e+00, -4.2684e+00, -7.3249e-01,  ..., -2.3027e-01,
           7.9486e-01, -8.9392e-02]],

        [[-6.6308e+00,  4.2375e+00,  5.8601e-02,  ...,  2.9390e-01,
          -2.1231e-01, -9.9223e-01],
         [-2.2451e-01, -3.5607e+00, -1.6302e+00,  ...,  3.2522e-02,
           1.0210e+00,  1.2264e+00],
         [ 2.6609e+00, -6.4346e-01, -1.7082e+00,  ..., -8.0307e-02,
           2.7711e-01, -1.7767e+00],
         ...,
         [-6.6873e-01, -4